In [1]:
from model_phantom_DB import *
from linares_plot import * 
import itertools

%matplotlib inline
%config InlineBackend.figure_format = 'png' ## 'svg'

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
import pandas as pd

For all the matrice data : each row is one trial, each column is one neuron. 
For the same row the trial data may not be collected on the same trial since data might are collected on different days.

cuerate: firing rate during 1st visual stimuli(cue) presentation.

cuerate_stim: firing rate during 1st cue presentation with NB stimulation. 

cdrate: firing rate during the 1st delay after 1st cue presentation. 

cdrate_stim: firing rate during the 1st delay after cue presentation  with NB stimulation.  
 
samplerate: firing rate during 2nd visual stimuli(sample) presentation. 

samplerate_stim: firing rate during sample presentation with NB stimulation. 

sdrate: firing rate during the delay after sample presentation. 

sdrate_stim: firing rate during the delay after sample presentation with NB stimulation. 

g1: location of 1st visual stimuli(cue)
 value -1 means no visual stimuli presented.
 value [1 2 3 4 5] are  relative locaions, such as [0 45 90 135 180], or [ 45 90 135 180 225], etc.
 The real locations depended on neurons' receptive field.
 According to our behavioral paradigm, the cue will only be presented on locaion [1 5]. that is two opposite locations, such as [0 180], [45 225], etc.
 For each neuron, thoes value([1:5]) refer to the same locations.

g2: location of 2nd visual stimuli(sample)
  Sample can show up in all five possible locations([1:5]) or not presented(-1).

g3: task type for each trial, eg. saccade to 1st (value 1), or saccade to 2nd(value 2).


In [2]:
from scipy.io import loadmat 
file = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Classifiy_data_odrdistvar_allNeurons_reverse.matlab'
data = loadmat(file)
data.keys()

dict_keys(['__version__', 'cdrate_stim', 'cdrate', '__globals__', 'cuerate', 'g3', 'sdrate_stim', 'cuerate_stim', 'g2_stim', '__header__', 'samplerate_stim', 'g3_stim', 'g1', 'sdrate', 'samplerate', 'g2', 'g1_stim'])

In [3]:
rate_d1_off = data['cdrate']
positions_stim_off= data['g1'] #1, 5, -1 (entiendo que 1 es RF y 5 es opposite, -1 es no estímulo)

rate_d1_on = data['cdrate_stim']
positions_stim_on= data['g1_stim'] #1, 5, -1 (entiendo que 1 es RF y 5 es opposite, -1 es no estímulo)

In [12]:
### neuron 1 off
Neuron = 0

n1_rates = rate_d1_off[:,Neuron]
n1_positions = positions_stim_off[:,Neuron]

#just get the 1 and 5 (RF and oppoiste)
n1_y = n1_positions[n1_positions>0]
n1_x = n1_rates[n1_positions>0]


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


In [13]:
y_off = label_binarize(n1_y, classes=[5,1]) #matrix (1,0,0,...,0)
n_classes = y_off.shape[1]
X_off =np.reshape(n1_x, (len(n1_x),1))